<a href="https://colab.research.google.com/github/shake/colab-Llama-2-ipynb/blob/main/phi2_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-requisites and Model Load

[Optimizing Phi-2: A Deep Dive into Fine-Tuning Small Language Models](https://medium.com/@yernenip/optimizing-phi-2-a-deep-dive-into-fine-tuning-small-language-models-9d545ac90a99)
Before we load the model, we will need to install the packages mentioned below. These packages do not come out of the box with Google Colab.

We also need to ensure that the correct runtime (for GPU) is selected. You can do this by clicking on `Runtime-->Change runtime type` in the File Menu above. For my project, I picked the T4 GPU, which comes with 16GB of CPU and GPU RAM.

In [ ]:
#Check the system specs
!nvidia-smi

In [1]:
#Install the required packages for this project
%%capture
!pip install einops datasets bitsandbytes accelerate peft flash_attn
!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers
!pip install --upgrade torch


## Login to Hugging Face
We will login to Hugging Face, so we can save the updated model weights when training is done. Make sure to use an access key that has write permissions. You can create one from the following location.

https://huggingface.co/settings/tokens

In [2]:
# import 密钥
%%capture
from google.colab import userdata
hf_token = userdata.get('huggingface')
!git config --global credential.helper store
!huggingface-cli login --token $hf_token --add-to-git-credential

### Loading the Microsoft Phi-2 Model
The Phi-2 model is available on Hugging Face. You can read the details of it from https://huggingface.co/microsoft/phi-2
I am also loading the model in `4-bit` which is the "Quantization" part of QLORA. The memory footprint of this is much smaller then the default.
Apart from loading the model, we will also setup the tokenizer and ensure the proper settings.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig

model_name = "microsoft/phi-2"
# Configuration to load model in 4-bit quantized
bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type='nf4',
                                bnb_4bit_compute_dtype='float16',
                                #bnb_4bit_compute_dtype=torch.bfloat16,
                                bnb_4bit_use_double_quant=True)


#Loading Microsoft's Phi-2 model with compatible settings
model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto',
                                             quantization_config=bnb_config,
                                             attn_implementation="flash_attention_2",
                                             trust_remote_code=True)

# Setting up the tokenizer for Phi-2
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          add_eos_token=True,
                                          trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = "left"


In [4]:
print(f"Memory footprint: {model.get_memory_footprint() / 1e9} GB")

Memory footprint: 1.792884736 GB


## Tokenize and Prep Dataset for Training

Next, we will load the WebGLM dataset. This dataset is a web-enhanced QA system based on General Language Model (GLM). You can find more information about this at https://huggingface.co/datasets/THUDM/webglm-qa.

I am taking a slice of data, as we will train our model for two epochs.

In [5]:
from datasets import load_dataset

#Load a slice of the WebGLM dataset for training and merge validation/test datasets
train_dataset = load_dataset("THUDM/webglm-qa", split="train[5000:10000]")
test_dataset = load_dataset("THUDM/webglm-qa", split="validation+test")

print(train_dataset)
print(test_dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['question', 'answer', 'references'],
    num_rows: 5000
})
Dataset({
    features: ['question', 'answer', 'references'],
    num_rows: 1400
})


In [6]:
#Function that creates a prompt from instruction, context, category and response and tokenizes it
def collate_and_tokenize(examples):

    question = examples["question"][0].replace('"', r'\"')
    answer = examples["answer"][0].replace('"', r'\"')
    #unpacking the list of references and creating one string for reference
    references = '\n'.join([f"[{index + 1}] {string}" for index, string in enumerate(examples["references"][0])])

    #Merging into one prompt for tokenization and training
    prompt = f"""###System:
Read the references provided and answer the corresponding question.
###References:
{references}
###Question:
{question}
###Answer:
{answer}"""

    #Tokenize the prompt
    encoded = tokenizer(
        prompt,
        return_tensors="np",
        padding="max_length",
        truncation=True,
        ## Very critical to keep max_length at 1024.
        ## Anything more will lead to OOM on T4
        max_length=2048,
    )

    encoded["labels"] = encoded["input_ids"]
    return encoded

In [7]:
#We will just keep the input_ids and labels that we add in function above.
columns_to_remove = ["question","answer", "references"]

#tokenize the training and test datasets
tokenized_dataset_train = train_dataset.map(collate_and_tokenize,
                                            batched=True,
                                            batch_size=1,
                                            remove_columns=columns_to_remove)
tokenized_dataset_test = test_dataset.map(collate_and_tokenize,
                                          batched=True,
                                          batch_size=1,
                                          remove_columns=columns_to_remove)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

In [8]:
#Check if tokenization looks good
input_ids = tokenized_dataset_train[1]['input_ids']

decoded = tokenizer.decode(input_ids, skip_special_tokens=True)
print(decoded)

###System:
Read the references provided and answer the corresponding question.
###References:
[1] - Elevation/Evolution: If your mole becomes raised after being flat, or it changes over a short period of time.
[2] There are many reasons why raised moles occur, the main one being a healthy benign intradermal mole, which can be genetic, long standing, soft and sometimes wobbly to touch. They may lose colour or get darker with age. These types of moles should be monitored for drastic change, but generally aren't cause for concern.
[3] Moles can lighten or darken in color, and raise or flatten. Sometimes, moles can even disappear altogether.
[4] However, moles that change and become raised could be an indication of melanoma (as pictured above), and as mentioned previously, if a mole changes, seek advice from skin cancer specialist.
[5] Yes. In some cases, moles may lighten or completely disappear later in life. In some instances, this is the result of the body's immune system attacking the

# Training the model

We will be using LORA technique to train the model. This technique will significantly reduce the number of trainable parameters, giving better performance and memory utilization.

In [9]:
#Accelerate training models on larger batch sizes, we can use a fully sharded data parallel model.
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

In [10]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [ ]:
from peft import prepare_model_for_kbit_training

print_trainable_parameters(model)

#gradient checkpointing to save memory
model.gradient_checkpointing_enable()

# Freeze base model layers and cast layernorm in fp32
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)
print(model)

In [12]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
    'q_proj',
    'k_proj',
    'v_proj',
    'dense',
    'fc1',
    'fc2',
    ], #print(model) will show the modules to use
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

lora_model = get_peft_model(model, config)
print_trainable_parameters(lora_model)


lora_model = accelerator.prepare_model(lora_model)

trainable params: 23592960 || all params: 1544985600 || trainable%: 1.53


In [13]:
print_trainable_parameters(lora_model)

trainable params: 23592960 || all params: 1544985600 || trainable%: 1.53


### Training the Model and saving to Hub
This is where, we setup the training arguments. These arguments have been carefully selected to improve memory utilization and also help increase performance. I played around with these for a while, before finalizing the following arguments.

Finally, I am saving the model weights to HuggingFace Hub, so we do not loose out work. The training can run for several hours and I usually keep it running overnight.

In [13]:
import time
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',  # Output directory for checkpoints and predictions
    overwrite_output_dir=True, # Overwrite the content of the output directory
    per_device_train_batch_size=1,  # Batch size for training
    per_device_eval_batch_size=1,  # Batch size for evaluation
    gradient_accumulation_steps=5, # number of steps before optimizing
    gradient_checkpointing=True,   # Enable gradient checkpointing
    gradient_checkpointing_kwargs={"use_reentrant": False},
    warmup_steps=50,  # Number of warmup steps
    #max_steps=1000,  # Total number of training steps
    num_train_epochs=2,  # Number of training epochs
    #learning_rate=5e-5,  # Learning rate
    learning_rate=2e-5,
    weight_decay=0.01,  # Weight decay
    optim="paged_adamw_8bit", #Keep the optimizer state and quantize it
    fp16=True, #Use mixed precision training
    #For logging and saving
    logging_dir='./logs',
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,  # Limit the total number of checkpoints
    evaluation_strategy="steps",
    eval_steps=100,
    load_best_model_at_end=True, # Load the best model at the end of training
)

trainer = Trainer(
    model=lora_model,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_test,
    args=training_args,
)

#Disable cache to prevent warning, renable for inference
model.config.use_cache = False

start_time = time.time()  # Record the start time
trainer.train()  # Start training
end_time = time.time()  # Record the end time

training_time = end_time - start_time  # Calculate total training time

print(f"Training completed in {training_time} seconds.")

#Save model to hub to ensure we save our work.
lora_model.push_to_hub("phi-2-webglm-qlora",
                  use_auth_token=True,
                  commit_message="Training Phi-2",
                  private=True)


#Terminate the session so we do not incur cost
from google.colab import runtime
runtime.unassign()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

# 保存模型到huggingface

In [ ]:
#Save model to hub to ensure we save our work.
lora_model.push_to_hub("phi2-webglm-qlora",
                  use_auth_token=True,
                  commit_message="Training Phi-2",
                  private=True)

## Run Inference

**Note**: Ensure to stop your session and reconnect and reload the model before running the code below.

First we will run inference without the trained weights and check the output.

In [ ]:
#Setup a prompt that we can use for testing

new_prompt = """###System:
Read the references provided and answer the corresponding question.
###References:
[1] For most people, the act of reading is a reward in itself. However, studies show that reading books also has benefits that range from a longer life to career success. If you’re looking for reasons to pick up a book, read on for seven science-backed reasons why reading is good for your health, relationships and happiness.
[2] As per a study, one of the prime benefits of reading books is slowing down mental disorders such as Alzheimer’s and Dementia  It happens since reading stimulates the brain and keeps it active, which allows it to retain its power and capacity.
[3] Another one of the benefits of reading books is that they can improve our ability to empathize with others. And empathy has many benefits – it can reduce stress, improve our relationships, and inform our moral compasses.
[4] Here are 10 benefits of reading that illustrate the importance of reading books. When you read every day you:
[5] Why is reading good for you? Reading is good for you because it improves your focus, memory, empathy, and communication skills. It can reduce stress, improve your mental health, and help you live longer. Reading also allows you to learn new things to help you succeed in your work and relationships.
###Question:
Why is reading books widely considered to be beneficial?
###Answer:
"""


In [ ]:
inputs = tokenizer(new_prompt, return_tensors="pt",
                   return_attention_mask=False,
                   padding=True, truncation=True)

inputs.to('cuda')

outputs = model.generate(**inputs, repetition_penalty=1.0,
                              max_length=1000)
result = tokenizer.batch_decode(outputs, skip_special_tokens=True)

print(result)

Next, lets run the model with lora config and check inference on it.

In [ ]:
from peft import PeftModel, PeftConfig

#Load the model weights from hub
model_id = "praveeny/phi2-webglm-qlora"
trained_model = PeftModel.from_pretrained(model, model_id)

#Run inference
outputs = trained_model.generate(**inputs, max_length=1000)
text = tokenizer.batch_decode(outputs,skip_special_tokens=True)[0]
print(text)